<a href="https://colab.research.google.com/github/MikiMassand/SL-GenAI-course-4/blob/main/Stable_Diffusion2_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stable diffusion Model Tranforming image using text prompt
- Able to load file from URL, or from folder
- Able adjust the strength and other parameters
- Able to generate caption


In [ ]:
!pip install --quiet --upgrade diffusers transformers  scipy ftfy
!pip install --quiet --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 8.9 MB/s eta 0:00:00


In [2]:
import torch
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionDepth2ImgPipeline
import urllib.parse as parse
import os

In [3]:
# load the model
model_id = "stabilityai/stable-diffusion-2-depth"

pipeline = StableDiffusionDepth2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline = pipeline.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

depth_estimator/config.json:   0%|          | 0.00/9.96k [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def check_url(string):
  try:
    result = parse.urlparse(string)
    return all([result.scheme, result.netloc, result.path])
  except:
    return False

In [5]:
def load_image(image_path):
  if check_url(image_path):
    return Image.open(requests.get(image_path, stream=True).raw)
  elif os.path.exists(image_path):
    return Image.open(image_path)

In [6]:
# load image
img = load_image("https://images.unsplash.com/photo-1465101162946-4377e57745c3?q=80&w=3878&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D")
img = img.resize((512, 512))

In [7]:
prompt = "A serene sunset over a calm lake"
n_prompt = "no buildings, no people"

# strength (0 to 1)
# num_inference_steps (1 to 50)
transformed_img = pipeline(prompt=prompt, image=img, negative_prompt=n_prompt,
                           strength=0.7, num_inference_steps=30).images[0]


transformed_img.save("transformed_img.png")

  0%|          | 0/21 [00:00<?, ?it/s]

In [8]:
for strength in [0.1, 0.4, 0.6, 0.8]:
  transformed_img = pipeline(prompt=prompt, image=img, negative_prompt=n_prompt,
                           strength=strength, num_inference_steps=30).images[0]
  transformed_img.save(f"transformed_img_{strength}.png")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

Image captioning

In [10]:
from transformers import BlipProcessor, BlipForConditionalGeneration

from PIL import Image
import requests

In [11]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [19]:
#image = Image.open('/content/monalisa.jpeg')
image = Image.open('/content/transformed_img.png')

# generate caption

inputs = processor(image, return_tensors="pt")
out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)

print("Generated Caption:", caption)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Caption: a tree with the moon in the background


SyntaxError: invalid syntax (<ipython-input-15-216b0abe9273>, line 1)

In [18]:
# prompt: show current folder path and contents

import os

def show_current_folder_contents():
  """Shows the current folder path and its contents."""
  current_path = os.getcwd()
  print(f"Current folder path: {current_path}")
  print("Contents:")
  for item in os.listdir(current_path):
    print(item)

show_current_folder_contents()

Current folder path: /content
Contents:
.config
transformed_img_0.4.png
transformed_img_0.6.png
transformed_img.png
transformed_img_0.8.png
transformed_img_0.1.png
sample_data
